# 2. Practical Part

## 1. Ridge Regression

In [1]:
import numpy as np

def ridge_regression(weight, bias, weightedDecay, stepSize, numberSteps, data, target):
    '''
    weightedDecay: λ
    stepSize: η
    x: data
    t: target
    
    '''
    for i in range(0, numberSteps):
        #gradient of a scalar x
        grad = np.inverse(np.dot(data.T , data)) *data*(bias-target) 
        #regularization lambda
        regGrad = weightedDecay * np.dot(data.T, data)
        
        regGrad = regGrad + grad
        weight = weight - stepSize*regGrad
    return w


     

## 2. Draw Dn from h(x)

In [8]:
x = np.random.uniform(-5,5,15)
hX = np.sin(x) + 0.3*x -1
dataDn = [x, hX]
print(dataDn)


[array([ 2.15970438, -2.38770133,  2.6684112 , -3.97092675, -4.52853786,
        0.13972784, -3.63939382,  4.92175453,  1.63107077,  2.16477458,
       -1.21866007, -1.59949983, -2.3325823 , -1.28242967,  2.10007598]), array([ 0.47945902, -2.40079123,  0.25624383, -1.45379622, -1.37541442,
       -0.81880803, -1.61432342, -0.50163662,  0.48750528,  0.47815314,
       -2.30423607, -2.47943803, -2.42337915, -2.34343855,  0.4931938 ])]


##  3. Train without regularization, $\lambda$ = 0
Plot h(x), Dn, and our prediction function $f(X) = w^T X + b $

## 4. Part 3 revisited, with difference lamda values
Plot prediction with $W$ regularized by $\lambda$ $\lVert w \rVert^2$
Choose $\lambda$ intermediate and large

## 5. Sample $D_{test}$ from $h(X)$
Use $\lambda$ = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100]

Plot $\lambda$  on x-axis, avg((f(X) - target)^2)

## 6. Use kernel $\phi(x)$
set $\lambda = 0.01$

Try different degrees of polynomials


## 7. Comment on $l$, empirical risk ($D_n$), true risk($D_{test}$)